In [1]:
import torch
import torchvision.transforms as transforms
from LookGenerator.datasets.tps_dataset import ShirtsDataset
from torch.utils.data import DataLoader
from LookGenerator.networks.bpgm.model.models import BPGM
from LookGenerator.networks.bpgm.train_bpgm import train_bpgm
import torch.nn.functional as F
import numpy as np
import os
import PIL.Image as Image
import matplotlib.pyplot as plt
from LookGenerator.networks_training.utils import check_path_and_creat
from LookGenerator.networks.utils import save_model, load_model

In [2]:
transform = transforms.Compose([
    transforms.Resize((256,192)),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])
mask_transform = transforms.Compose([
    transforms.Resize((256,192)),
])


In [3]:
batch_size = 64
num_workers = 0
pin_memory = True

In [4]:
dataset = ShirtsDataset(
    root=r"C:\Users\DenisovDmitrii\Desktop\zalando-hd-resize\train",
    transform=transform,
    transform_mask=mask_transform
)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, pin_memory=pin_memory, num_workers=num_workers)

In [5]:
save_directory = r'C:\Users\DenisovDmitrii\OneDrive - ITMO UNIVERSITY\peopleDetector\tps\weights\test'
check_path_and_creat(save_directory)

True

In [6]:
for data in dataloader:
    break

C:\Users\DenisovDmitrii\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [8]:

model = BPGM(device=device)

initialization method [normal]
initialization method [normal]


In [9]:
loss = train_bpgm(dataloader=dataloader,
                  model=model, device=device, epochs=1)

C:\Users\DenisovDmitrii\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\DenisovDmitrii\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 182/182 [15:58<00:00,  5.27s/it]

[]


In [10]:
save_model(model.to('cpu'), path=f"{save_directory}\\epoch_02.pt")

# тест

In [24]:
test_dataset = ShirtsDataset(
    root=r"C:\Users\DenisovDmitrii\Desktop\zalando-hd-resize\test",
    transform=transform,
    transform_mask=mask_transform
)

In [11]:
topil = transforms.ToPILImage()

In [26]:
testdataloader = DataLoader(test_dataset, batch_size=1, pin_memory=True)

In [8]:
weights_dir = r"C:\Users\DenisovDmitrii\OneDrive - ITMO UNIVERSITY\peopleDetector\tps\weights\test\epoch_00.pt"

In [9]:
model = load_model(model, weights_dir)

In [36]:
count = 0
for data in testdataloader:
    mask = data['segmentation'].to(device)
    shirt = data['shirt'].to(device)
    shirt_mask = data['shirt_mask'].to(device)

    model = model.to(device)
    theta = model(mask, shirt)

    warped = F.grid_sample(shirt, theta, padding_mode='border', align_corners=True).to('cpu')

    warped_mask = F.grid_sample(shirt_mask, theta, padding_mode='border', align_corners=True).to('cpu')
    warped_mask = topil(warped_mask[0])
    warped_mask.show()
    warped = topil(warped[0]/2+0.5)
    warped.show()
    count += 1
    if count == 4:
        break

подготовка

In [12]:
path_people = r'C:\Users\DenisovDmitrii\Desktop\forEncoderNew\train\imageWithNoCloth'

In [13]:
cloud_people = os.listdir(path_people)

In [14]:
len(cloud_people)

11647

In [15]:
# ИСПОЛЬЗОВАТЬ batch_size=1
count = 0
for index, data in enumerate(dataloader):

    cloud_person = Image.open(path_people + "\\" + cloud_people[index])
    cloud_person = np.array(cloud_person)

    mask = data['segmentation'].to(device)
    shirt = data['shirt'].to(device)
    shirt_mask = data['shirt_mask'].to(device)

    model = model.to(device)
    theta = model(mask, shirt)

    warped = F.grid_sample(shirt, theta, padding_mode='border', align_corners=True).to('cpu')

    warped_mask = F.grid_sample(shirt_mask, theta, padding_mode='border', align_corners=True).to('cpu')
    warped = warped / 2 + 0.5
    wc = warped * warped_mask
    # wc = transforms.Resize((256 + 256 // 2, 192 + 192 // 2))(wc)
    # wc = transforms.CenterCrop((256, 192))(wc)
    warped_mask = np.array(topil(warped_mask[0]))
    # warped = np.array(topil(warped[0]))
    wc = np.array(topil(wc[0]))
    # wc = np.concatenate((warped[:,:,0] * warped_mask, warped[:,:,1] * warped_mask, warped[:,:,2] * warped_mask), axis=2)
    # wc = warped * warped_mask
    # wc += warped[:,:,1] * warped_mask
    # wc += warped[:,:,2] * warped_mask
    print(wc.shape, cloud_person.shape)
    person = cloud_person + wc

    Image.fromarray(person, 'RGB').save(path_people + "2\\" + cloud_people[index])
    count += 1
    if count == 4:
        break

(256, 192, 3) (256, 192, 3)
(256, 192, 3) (256, 192, 3)
(256, 192, 3) (256, 192, 3)
(256, 192, 3) (256, 192, 3)
